In [56]:
from sklearn.neighbors import KNeighborsClassifier
import warnings
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    equivalence_classes = calculate_equivalence_classes(data,feature_columns)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    entropy_DB = 0
    rule_confidences=0
    total_samples=0
    total_rule=0
    for equivalence_class in equivalence_classes.values():
        if len(equivalence_class)>total_records*0.0:
            total_samples=total_samples+len(equivalence_class)
            total_rule=total_rule+1
            a=[]
            for key_decision in equivalence_classes_dec.keys():
                x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
                y = 1-len(x) / len(equivalence_class)
                a.append(y)
            max_rule = min(a)
            rule_confidences=rule_confidences+max_rule

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            if len(x)>0:
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    # rule_confidences = np.array(rule_confidences)
    # count = len(rule_confidences)
    if total_samples/total_records<0.9:
        return (1,)

    avg_confidence = rule_confidences / total_rule +0.01*(-entropy_DB)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    random.seed(6)
    # create individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof
    # return hof

In [57]:
def bestIndividual_KNN(hof,df):
    k = 4
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('../../data/Zoo.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features

In [58]:
from sklearn.model_selection import StratifiedKFold
def bestIndividual_SVM(hof, df):
    def SVM(trainX, trainY, testX, testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.1, max_iter=500)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY) / testY.shape[0]
        return precision

    maxaccuracy = 0.0
    for ind in hof:
        df = pd.read_csv('../../data/Zoo.csv')
        feature_columns = [col for col in df.columns if col not in ['id', 'target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y = df["target"]
        X = df[selected_features]

        # Use stratified k-fold cross-validation
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        for train_index, test_index in skf.split(X, Y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index]
            trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
            score = SVM(trainX, trainY, testX, testY)
            scores.append(score)
        avg_score = np.mean(scores)

        if avg_score > maxaccuracy:
            maxaccuracy = avg_score
            best_feature = selected_features

    return maxaccuracy, best_feature

In [59]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

def bestIndividual_DT(hof, df):
    for ind in hof:
        df = pd.read_csv('../../data/Zoo.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        X = df[selected_features]
        Y = df['target']
        dt = DecisionTreeClassifier()
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            dt.fit(X_train, y_train)

            y_pred = dt.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)

        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    return mean_accuracy

In [60]:
if __name__ == '__main__':
    # get dataframe path, population number and generation number from command-line argument
    # dataframePath, n_pop, n_gen = getArguments()
    n_pop=50
    n_gen=30
    # read dataframe from csv
    df = pd.read_csv('../../data/Zoo.csv')
    # encode labels column to numbers
    y = df['target']
    feature_columns = [ col for col in df.columns if col not in ['id','target']]
    X = df[feature_columns]
    # for col in feature_columns:
    #     df[col] = pd.qcut(df[col], q=5, labels=False, duplicates='drop')
    # get accuracy with all features
    individual = [1 for i in range(len(X.columns))]
    getFitness(df,'target',individual)
    print("Accuracy with all features: \t" +
          str(getFitness(df,'target',individual)) + "\n")

    # apply genetic algorithm
    hof = geneticAlgorithm(df,'target',n_pop, n_gen)

Accuracy with all features: 	(0.0,)

gen	nevals	avg	min
0  	50    	0  	0  
1  	29    	0  	0  
2  	22    	0  	0  
3  	30    	0  	0  
4  	27    	0  	0  
5  	33    	0  	0  
6  	32    	0  	0  
7  	27    	0  	0  
8  	25    	0  	0  
9  	27    	0  	0  
10 	34    	0  	0  
11 	30    	0  	0  
12 	29    	0  	0  
13 	37    	0  	0  
14 	31    	0  	0  
15 	25    	0  	0  
16 	26    	0  	0  
17 	32    	0  	0  
18 	26    	0  	0  
19 	33    	0  	0  
20 	30    	0  	0  
21 	26    	0  	0  
22 	28    	0  	0  
23 	32    	0  	0  
24 	30    	0  	0  
25 	28    	0  	0  
26 	37    	0  	0  
27 	28    	0  	0  
28 	33    	0  	0  
29 	38    	0  	0  
30 	26    	0  	0  
Top 1 solution: [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1]
Fitness: 0.0



In [61]:
accuracy_KNN,individual = bestIndividual_KNN(hof,df)
print("-----------KNN---------------")
print(accuracy_KNN)

-----------KNN---------------
0.8109090909090909


In [62]:
print('------------特征-----------------')
print(individual)
print(len(individual))

------------特征-----------------
['aquatic', 'predator', 'toothed', 'venomous', 'fins', 'tail', 'catsize']
7


In [63]:
accuracy_SVM,individual = bestIndividual_SVM(hof,df)
print("-----------SVM---------------")
print(accuracy_SVM)

-----------SVM---------------
0.7627272727272727


In [64]:
print("-----------DT---------------")
accuracy_DT = bestIndividual_DT(hof,df)
print(accuracy_DT)

-----------DT---------------
0.7809090909090909
